In [508]:
#data_path_all is the Anndata object containing all cells
#data_path_meta is the Anndata object conta 
#assign_path is the metadata
#metacell_types_path gives the cell types of the metacells
#benchmarkindices_path imports 
data_path_all = '/home/labs/shlush/shared/runs_analysis/Metacells_Models/Nili_Human/data_July_2023/all_cells_fil2_500000_cdata_output_with_cell_type_exp_indiv_new.h5ad'
data_path_meta = '/home/labs/shlush/shared/runs_analysis/Metacells_Models/Nili_Human/data_July_2023/all_cells_fil2_500000_mdata_output_with_type_new.h5ad'
assign_path = '/home/labs/shlush/shared/runs_analysis/Metacells_Models/Nili_Human/cell_metadata'
metacell_types_path = '/home/labs/shlush/tomerlev/LOY/metacell_types-2023-03-16.csv'
benchmarkindices_path='/home/labs/shlush/noac/workspace/ISSI_2023/indices.csv'
all_cells = ad.read_h5ad(data_path_all)
metacells = ad.read_h5ad(data_path_meta)
benchmarkindices=pd.read_csv(benchmarkindices_path)
benchmarkindices=list(benchmarkindices["indices"])

import numpy as np
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
import scipy
import statistics
import math

In [228]:
#Parameters
UMIbenchmark = 2000

In [ ]:
#Find out cell type
all_cells.obs.head()
cells_to_meta_df = pd.DataFrame()
cells_to_meta_df['cnames'] = all_cells.obs['cnames']
cells_to_meta_df['metacell'] = all_cells.obs['metacell']
cells_to_meta_df.head()

metacell_label_df = pd.read_csv(metacell_types_path)
meta_type_dict = {}
for _, row in metacell_label_df.iterrows():
    meta_type_dict[row['metacell']] = row['cell_type']
    
cell_types = []
for _, row in cells_to_meta_df.iterrows():
    try:
        cell_types.append(meta_type_dict[row['metacell']])
    except:
        cell_types.append('unassigned')
cells_to_meta_df['cell_type'] = cell_types
cells_to_meta_df.tail()

In [230]:
#input genes from a chromosome as a list into entries. Make sure to get rid of genes you don't want (in our case it was SLC25A6 and CD99)
#input gene "synonyms" (e.g. ACOD1 and IRG1)


all_cells_genes = all_cells.var_names
metacell_genes = metacells.var_names
meta_sm = metacells.X
ygenes1=['LOC124905300', 'PLCXD1', 'GTPBP6', 'LINC00685', 'PPP2R3B', 'LOC102724521', 'SHOX', 'CRLF2', 'CSF2RA', 'LOC124905238', 'MIR3690', 'IL3RA', 'LOC101928032', 'LOC105373102', 'LINC00106', 'ASMTL-AS1', 'ASMTL', 'P2RY8', 'AKAP17A', 'ASMT', 'LINC02968', 'LOC105379413', 'LOC107985677', 'DHRSX', 'LOC124905239', 'ZBED1', 'LOC101928092', 'MIR6089', 'CD99P1', 'LINC00102', 'XGY2', 'SRY', 'RPS4Y1', 'ZFY', 'ZFY-AS1', 'LINC00278', 'TGIF2LY', 'LOC105377227', 'MIR9985', 'PCDH11Y', 'TTTY23B', 'TSPY2', 'FAM197Y9', 'LINC00280', 'LOC107987353', 'TTTY1B', 'TTTY2B', 'TTTY21B', 'TTTY7', 'TTTY8B', 'AMELY', 'TBL1Y', 'PRKY', 'LOC107987338', 'TTTY16', 'LOC107987339', 'TTTY12', 'LOC105377229', 'LOC105377231', 'LOC105377232', 'LOC107987354', 'LINC00279', 'LOC107987340', 'TTTY18', 'LOC105377234', 'TTTY19', 'LOC105377235', 'TTTY11', 'LOC105379264', 'LOC105379265', 'LOC105379266', 'LOC105379267', 'RBMY1A3P', 'TTTY20', 'TSPY4', 'FAM197Y8', 'TSPY8', 'FAM197Y7', 'FAM197Y6', 'TSPY3', 'FAM197Y5', 'TSPY1', 'FAM197Y4', 'TSPY9', 'FAM197Y3', 'FAM197Y2', 'TSPY10', 'RBMY3AP', 'TTTY8', 'TTTY7B', 'TTTY21', 'TTTY2', 'TTTY1', 'LOC107987342', 'TTTY22', 'TTTY23', 'LOC105379273', 'LOC107987343', 'GYG2P1', 'LOC112268311', 'TTTY15', 'USP9Y', 'DDX3Y', 'LOC112268312', 'UTY', 'MIR12120', 'TMSB4Y', 'VCY', 'VCY1B', 'NLGN4Y', 'NLGN4Y-AS1', 'LOC107987355', 'LOC124900508', 'FAM41AY1', 'LOC105377217', 'FAM224B', 'LOC105377218', 'LOC124905302', 'XKRY', 'CDY2B', 'CDY2A', 'XKRY2', 'LOC124905303', 'LOC105377219', 'FAM224A', 'LOC105377220', 'FAM41AY2', 'LOC124905304', 'HSFY1', 'TTTY9B', 'TTTY9A', 'HSFY2', 'LOC124905305', 'LOC124905307', 'TTTY14', 'LOC124905306', 'BCORP1', 'LOC102723934', 'LOC105377223', 'TXLNGY', 'KDM5D', 'LOC105377224', 'TTTY10', 'EIF1AY', 'RPS4Y2', 'LOC105377225', 'LOC107987346', 'LOC107987347', 'PRORY', 'RBMY2EP', 'RBMY1B', 'RBMY1A1', 'TTTY13', 'LOC105377236', 'LOC105377237', 'RBMY1D', 'RBMY1E', 'LOC107987356', 'PRY2', 'LOC102725532', 'TTTY6B', 'RBMY1F', 'TTTY5', 'RBMY2FP', 'LOC100652931', 'RBMY1J', 'LOC101929148', 'TTTY6', 'PRY', 'LOC107987357', 'LOC107987348', 'TTTY17A', 'TTTY4', 'BPY2', 'LOC105377238', 'DAZ1', 'DAZ2', 'LOC105377239', 'LOC124900506', 'REREP1Y', 'LOC107987349', 'TTTY3B', 'CDY1B', 'GOLGA2P2Y', 'LOC107987350', 'TTTY17B', 'TTTY4B', 'BPY2B', 'LOC105377241', 'DAZ3', 'DAZ4', 'LOC105377242', 'BPY2C', 'TTTY4C', 'TTTY17C', 'SEPTIN14P23', 'GOLGA2P3Y', 'CSPG4P1Y', 'CDY1', 'TTTY3', 'LOC107987352', 'REREP2Y', 'LOC124900507', 'LOC105377244', 'SPRY3', 'VAMP7', 'IL9R', 'WASIR1', 'DDX11L16']
ygenes2=['PLCXD1', 'GTPBP6', 'LINC00685', 'PPP2R3B', 'SHOX', 'CRLF2', 'CSF2RA', 'MIR3690', 'IL3RA', 'LINC00106', 'ASMTL-AS1', 'ASMTL', 'P2RY8', 'AKAP17A', 'ASMT', 'DHRSX', 'ZBED1', 'MIR6089', 'CD99P1', 'LINC00102', 'XGY2', 'SRY', 'RPS4Y1', 'ZFY', 'ZFY-AS1', 'LINC00278', 'TGIF2LY', 'MIR9985', 'PCDH11Y', 'TTTY23B', 'TSPY2', 'FAM197Y9', 'LINC00280', 'TTTY1B', 'TTTY2B', 'TTTY21B', 'TTTY7', 'TTTY8B', 'AMELY', 'TBL1Y', 'PRKY', 'TTTY16', 'TTTY12', 'LINC00279', 'TTTY18', 'TTTY19', 'TTTY11', 'RBMY1A3P', 'TTTY20', 'TSPY4', 'FAM197Y8', 'TSPY8', 'FAM197Y7', 'FAM197Y6', 'TSPY3', 'FAM197Y5', 'TSPY1', 'FAM197Y4', 'TSPY9', 'FAM197Y3', 'FAM197Y2', 'TSPY10', 'RBMY3AP', 'TTTY8', 'TTTY7B', 'TTTY21', 'TTTY2', 'TTTY1', 'TTTY22', 'TTTY23', 'GYG2P1', 'TTTY15', 'USP9Y', 'DDX3Y', 'UTY', 'MIR12120', 'TMSB4Y', 'VCY', 'VCY1B', 'NLGN4Y', 'NLGN4Y-AS1', 'FAM41AY1', 'FAM224B', 'XKRY', 'CDY2B', 'CDY2A', 'XKRY2', 'FAM224A', 'FAM41AY2', 'HSFY1', 'TTTY9B', 'TTTY9A', 'HSFY2', 'TTTY14', 'BCORP1', 'TXLNGY', 'KDM5D', 'TTTY10', 'EIF1AY', 'RPS4Y2', 'PRORY', 'RBMY2EP', 'RBMY1B', 'RBMY1A1', 'TTTY13', 'RBMY1D', 'RBMY1E', 'PRY2', 'TTTY6B', 'RBMY1F', 'TTTY5', 'RBMY2FP', 'LOC100652931', 'RBMY1J', 'LOC101929148', 'TTTY6', 'PRY', 'TTTY17A', 'TTTY4', 'BPY2', 'DAZ1', 'DAZ2', 'TTTY3B', 'CDY1B', 'GOLGA2P2Y', 'TTTY17B', 'TTTY4B', 'BPY2B', 'DAZ3', 'DAZ4', 'BPY2C', 'TTTY4C', 'TTTY17C', 'SEPTIN14P23', 'GOLGA2P3Y', 'CSPG4P1Y', 'CDY1', 'TTTY3', 'SPRY3', 'VAMP7', 'IL9R', 'WASIR1', 'DDX11L16']
entries=(list(set(ygenes1)|set(ygenes2)))

synonyms = ["TSPY9P"]
og_chry_gene_names = set(entries)


not_included = set([])
included = set([])

#detect if chromsome genes are in the anndata gene list
#chry_gene_names is the y chromsome gene names in the anndata gene list
#subset_df is the dataframe containing gene expression of Y chromosome genes
for gene in og_chry_gene_names:
    if gene in all_cells_genes:
        included.add(gene)  # if y-chr gene, put in included
    else: not_included.add(gene) # else, not_included -- doesn't add not included synonyms

for gene in synonyms:
    if gene in all_cells_genes:
        included.add(gene)

chry_gene_names = included 

subset = all_cells[all_cells.obs_names,list(chry_gene_names)] # gives subset of anndata object including only "included" y-cgenes

subset_df = pd.DataFrame.sparse.from_spmatrix(subset.X, columns=subset.var_names)
subset_df.index = subset.obs_names


In [231]:
#creating a DF with UMI and total UMi for each Y gene from every cell
cells_counts_df = subset_df

#adding all UMI's per Y gene, concat to different dataframe
umi_sum=list(cells_counts_df.sum())
umi_sum=pd.DataFrame(umi_sum)
umi_sum=umi_sum.transpose()


umi_sum.columns=cells_counts_df.columns
cells_counts_df=pd.concat([cells_counts_df,umi_sum])
new_names={0:"Total UMI"}
cells_counts_df=cells_counts_df.rename(index=new_names)

temp_df=subset_df
cells_expressed=[(temp_df[col]>0.0).sum() for col in temp_df.columns]
cells_expressed=pd.DataFrame(cells_expressed)
cells_expressed=cells_expressed.transpose()
cells_expressed.columns=cells_counts_df.columns
cells_counts_df=pd.concat([cells_counts_df,cells_expressed])

new_names={0:"Number of Cells Expressing Gene"}
cells_counts_df=cells_counts_df.rename(index=new_names)
#cells_counts_df contains the gene expression data for all Y genes and has two additional rows, Total UMI of y genes and number of cells expressing the y gene

In [233]:
#creating a DF (alldf1) with gene expression and total UMI for each gene from every cell
all_cells
alldf=pd.DataFrame.sparse.from_spmatrix(all_cells.X, columns=all_cells.var_names)
alldf.index = subset.obs_names
alldf1 = alldf
umi_sum1=list(alldf1.sum())
umi_sum1=pd.DataFrame(umi_sum1)
umi_sum1=umi_sum1.transpose()

umi_sum1.columns=alldf1.columns
alldf1=pd.concat([alldf1,umi_sum1])
new_names={0:"Total UMI"}
alldf1=alldf1.rename(index=new_names)

In [234]:
#sexlist is list of sex for every cell, agelist is list for every age
#ydf is cells_counts_df but a column of Sex and Age for cells is added
#alldf1 is alldf but with Sex and Age with only Gene Expression
ydf=cells_counts_df
metadata_df = pd.read_csv(assign_path, sep=" ", header=0)
samplesex=metadata_df['sex']
sexlist=samplesex.to_list()
sexlist.append("Total UMI")
sexlist.append("Number of Cells Expressing Gene")
agelist=metadata_df['age'].to_list()
agelist.append("Total UMI")
agelist.append("Number of Cells Expressing Gene")
ydf["Sex"]=sexlist
ydf["Age"]=agelist
mydf=ydf[ydf["Sex"]=="male"]
fydf=ydf[ydf["Sex"]=="female"]
nydf=ydf[ydf["Sex"].isna()]
len(mydf.index.to_list())+len(fydf.index.to_list())+len(nydf.index.to_list())
sexlist1=sexlist
sexlist1.remove('Number of Cells Expressing Gene')

agelist1=agelist
agelist1.remove('Number of Cells Expressing Gene')
alldf1['Sex']=sexlist1
alldf1['Age']=agelist1
tempdf1=alldf1
tempdf1.drop('Sex', axis=1,inplace=True)
tempdf1.drop('Age', axis=1,inplace=True)

In [ ]:
#list of total umis per cell
totalumis=scipy.sparse.csr_matrix.sum(all_cells.X,axis=1)

totalumis=totalumis.tolist()

umisum=[]
for i in range (0,360810):
    umisum.append(totalumis[i][0])
    
tempydf=ydf
yumisum=list(tempydf.sum(axis=1))

yumisum.pop()
yumisum.pop()

#Calculating Y Score
yscorelist=[]
for i in range (0,360810):
    if totalumis[i][0]==0:
        yscorelist.append(0)
    else:
        yscorelist.append(yumisum[i]/totalumis[i][0])
        
cellids=tempdf1.index.tolist()
cellids.pop()

In [237]:
#return indices of cells to keep after filtering by UMI benchmark for every cell in keepindices
keepindices=[]
for i in range(0,len(totalumis)):
    if totalumis[i][0]>UMIbenchmark:
         keepindices.append(i)

#returns list of yscores of cells after filter in keepyscorelist
#returns list of cellids of cells after filter in keepcellids
keepyscorelist=[]
keepcellids=[]
for i in keepindices:
    keepyscorelist.append(yscorelist[i])
    keepcellids.append(cellids[i])

#yscoredf: Dataframe with each filtered cell and its Y score
yscoredf=pd.DataFrame(keepcellids,columns=["Cell ID"])
yscoredf['Y Score']=keepyscorelist

data = {
    'Cell ID': cellids,
    'Y Score': yscorelist
}

yscoredf = pd.DataFrame(data)

In [54]:
#return indices of male cells, female cells, and cells with unknown sex
maleindices=[]
femaleindices=[]
naindices=[]
for i in range (0,360810):
    if ydf["Sex"][i]=="male":
        maleindices.append(i)
    elif ydf["Sex"][i]=="female":
        femaleindices.append(i)
    else:
        naindices.append(i)

In [328]:
yscorelist=yscoredf['Y Score'].tolist()
LOYindex=[]
notLOYindex=[]
#returns indices of LOY and indices of not LOY cells
for i in range(0, len(yscoredf.index)):
    if yscorelist[i]<0.001:
        LOYindex.append(i)
    else:
        notLOYindex.append(i)

#Removes all female cells from the LOY and not LOY index lists and removes cells that don't pass the benchmark
LOYindex=list(set(LOYindex)&set(maleindices)&set(benchmarkindices))
notLOYindex=list(set(notLOYindex)&set(maleindices)&set(benchmarkindices))
#creates sparce matrices of male cells that pass the benchmark with LOY or not LOY
LOYsparse=all_cells[all_cells.obs_names[LOYindex],all_cells_genes]
notLOYsparse=all_cells[all_cells.obs_names[notLOYindex],all_cells_genes]
#turns said sparce matrices into dfs
LOYdf=pd.DataFrame.sparse.from_spmatrix(LOYsparse.X, columns=LOYsparse.var_names)
notLOYdf=pd.DataFrame.sparse.from_spmatrix(notLOYsparse.X, columns=notLOYsparse.var_names)
#returns umi totals for genes amongst LOY and not LOY cell groups
LOYdfsum=LOYdf.sum()
notLOYdfsum=notLOYdf.sum()

In [ ]:
#sum the total umi for LOY cells and total umi for not LOY cells. Used for normalization
LOYsum=sum([umisum[i] for i in range(0,len(umisum)) if i in LOYindex])
noLOYsum=sum([umisum[i] for i in range(0,len(umisum)) if i in noLOYindex])

In [ ]:
#graph of male vs female Y scores
fig, ax = plt.subplots(figsize=(12,9))
sns.kdeplot(np.array(yscorelist)[list(set(maleindices)&set(benchmarkindices))], shade=True, color='blue', label='Male', ax=ax)
sns.kdeplot(np.array(yscorelist)[list(set(femaleindices)&set(benchmarkindices))], shade=True, color='red', label='Female', ax=ax)
ax.legend()
ax.set_title('Male vs Female Cells Y-Scores')
plt.show()

In [287]:
#Identify 0s
zeroindex=[i for i in range(0,len(umisum)) if (umisum[i]==0)]

In [ ]:
#umisum is sum of umis per gene. Log normalizes
labels=ydf.columns
logLOY=[math.log(LOYumi[i]/LOYsum) for i in range(0,len(LOYumi))]
lognoLOY=[math.log(noLOYumi[i]/noLOYsum) for i in range(0,len(noLOYumi))]

In [361]:
#Identify genes with 0 UMIs in LOY or no LOY 
zeroLOY=[i for i in range(0, 33487) if LOYdfsum[i]==0]
zeronoLOY=[i for i in range(0, 33487) if notLOYdfsum[i]==0]
zeros=list(set(zeroLOY)|set(zeronoLOY))

In [458]:
#identify indices of y genes to be removed
yindices=[]
for i in range(0,33487):
    if all_cells_genes[i] in chry_gene_names:
        yindices.append(i)

In [ ]:
#remove index of genes with 0 total umi or are y genes. returns genes that have gene umi>0 and aren't y genes
notLOYdfindexlist=list(range(0,33487))
for i in list(set(zeros)|set(yindices)): 
    print(i)
    notLOYdfindexlist.remove(i)

In [ ]:
#Non log transformed plot without 0 UMI genes of UMI differential gene expression between LOY and non LOY
fig, ax = plt.subplots(figsize=(20,20))
sns.scatterplot(x=(notLOYdf.sum()[notLOYdfindexlist])/noLOYsumy, y=LOYdf.sum()[notLOYdfindexlist]/LOYsumy,color="blue") # Add labels to each point 
sns.regplot(x=notLOYdf.sum()[notLOYdfindexlist]/noLOYsumy, y=LOYdf.sum()[notLOYdfindexlist]/LOYsumy)
plt.xlabel('No LOY Gene Total UMI') 
plt.ylabel('LOY Gene Total UMI') 
plt.title('Removed 0 UMI LOY vs No LOY Gene Expression in Male Cells Benchmark 2000')
plt.savefig('LOY vs No LOY Gene Expression in Male Cells Benchmark 2000.png')
plt.show()

In [472]:
#log transform LOY
lognotLOY=[]
for i in list(notLOYdf.sum()[notLOYdfindexlist]/noLOYsumy):
    lognotLOY.append(math.log(i))
logLOY=[]
for i in list(LOYdf.sum()[notLOYdfindexlist]/LOYsumy):
    logLOY.append(math.log(i))

In [ ]:
#log transformed UMI LOY vs no LOY
fig, ax = plt.subplots(figsize=(20,20))
sns.scatterplot(x=np.array(lognotLOY), y=np.array(logLOY),color="blue") # Add labels to each point 
sns.regplot(x=np.array(lognotLOY), y=np.array(logLOY))
label=list(all_cells_genes[notLOYdfindexlist])
plt.xlabel('No LOY Gene Total UMI') 
plt.ylabel('LOY Gene Total UMI') 
plt.title('Log Transformed LOY vs No LOY Gene Expression in Male Cells Benchmark 2000')
plt.savefig('Log LOY vs No LOY Gene Expression in Male Cells Benchmark 2000.png')
plt.show()

In [ ]:
#Residual plot of log transformed UMIs
fig, ax = plt.subplots(figsize=(50,50))
model = sns.regplot(x=lognotLOY, y=logLOY, scatter=True, fit_reg=True)
x_regression = model.get_lines()[0].get_xdata()
y_regression = model.get_lines()[0].get_ydata()
plt.clf()
# Calculate residuals
residuals = np.array(logLOY) - np.interp(lognotLOY, x_regression, y_regression)

sns.residplot(x=np.array(lognotLOY), y=residuals, lowess=True, line_kws={'color': 'red'})

# # Adding labels and title
plt.xlabel("No LOY Gene Total UMI")
plt.ylabel("LOY Gene Total UMI Residuals")
label=list(all_cells_genes[notLOYdfindexlist])
for i, residual in enumerate(residuals):
    if abs(residual) > 0.5:
        plt.annotate(label[i], (np.array(lognotLOY)[i], residual))

plt.title('Large Residual Plot Log Transformed LOY vs No LOY Gene Expression in Male Cells Benchmark 2000')
plt.savefig('Large Residual Plot Log LOY vs No LOY Gene Expression in Male Cells Benchmark 2000.png')
plt.show()

In [134]:
#get unique cell type names, get list of cell types for all cells
celltypelist=list(np.unique(list(all_cells.obs["cell_type"])))
celltypelonglist=list(all_cells.obs["cell_type"])

In [148]:
#Get indices for every cell type
celltypeindex={}
for i in celltypelist:
    templist=[]
    for j in range(0,len(all_cells.obs.index)):
        if celltypelonglist[j]==i:
            templist.append(j)
    celltypeindex[i]=templist
        

In [ ]:
#Generate graphs of Total Umi of Genes in LOY vs not LOY groups for regular UMI, log transformed UMI, and residuals. 

yscorelist=yscoredf['Y Score'].tolist()
LOYindex=[]
notLOYindex=[]

for celltype in celltypelist:
    for i in range(0, len(yscoredf.index)):
        if yscorelist[i]<0.001:
            LOYindex.append(i)
        else:
            notLOYindex.append(i)
    LOYindex=list(set(LOYindex)&set(maleindices)&set(benchmarkindices)&set(celltypeindex[celltype]))
    notLOYindex=list(set(notLOYindex)&set(maleindices)&set(benchmarkindices)&set(celltypeindex[celltype]))
    LOYsparse=all_cells[all_cells.obs_names[LOYindex],all_cells_genes]
    notLOYsparse=all_cells[all_cells.obs_names[notLOYindex],all_cells_genes]
    LOYdf=pd.DataFrame.sparse.from_spmatrix(LOYsparse.X, columns=LOYsparse.var_names)
    notLOYdf=pd.DataFrame.sparse.from_spmatrix(notLOYsparse.X, columns=notLOYsparse.var_names)
    LOYdfsum=LOYdf.sum()
    notLOYdfsum=notLOYdf.sum()
    print(LOYindex)
    print(notLOYindex)
    zeroLOY=[i for i in range(0, 33487) if LOYdfsum[i]==0]
    zeronoLOY=[i for i in range(0, 33487) if notLOYdfsum[i]==0]
    zeros=list(set(zeroLOY)|set(zeronoLOY))
    
    notLOYdfindexlist=list(range(0,33487))
    for i in list(set(zeros)|set(yindices)): 
        notLOYdfindexlist.remove(i)
    print(celltype)
    if (notLOYdfindexlist!=[]):
        fig, ax = plt.subplots(figsize=(20,20))
        print(notLOYdfindexlist)
        print(notLOYdf.sum()[notLOYdfindexlist])
        sns.scatterplot(x=(notLOYdf.sum()[notLOYdfindexlist])/noLOYsumy, y=LOYdf.sum()[notLOYdfindexlist]/LOYsumy,color="blue") # Add labels to each point 
        sns.regplot(x=notLOYdf.sum()[notLOYdfindexlist]/noLOYsumy, y=LOYdf.sum()[notLOYdfindexlist]/LOYsumy)
        plt.xlabel('No LOY Gene Total UMI') 
        plt.ylabel('LOY Gene Total UMI') 
        plt.title(f'{celltype} Removed 0 UMI LOY vs No LOY Gene Expression in Male Cells Benchmark 2000')
        plt.savefig(f'{celltype} Removed 0 LOY vs No LOY Gene Expression in Male Cells Benchmark 2000.png')
        plt.show()

        lognotLOY=[]
        for i in list(notLOYdf.sum()[notLOYdfindexlist]/noLOYsumy):
            lognotLOY.append(math.log(i))

        logLOY=[]
        for i in list(LOYdf.sum()[notLOYdfindexlist]/LOYsumy):
            logLOY.append(math.log(i))

        fig, ax = plt.subplots(figsize=(20,20))
        sns.scatterplot(x=np.array(lognotLOY), y=np.array(logLOY),color="blue") # Add labels to each point 
        sns.regplot(x=np.array(lognotLOY), y=np.array(logLOY))
        label=list(all_cells_genes[notLOYdfindexlist])
        plt.xlabel('No LOY Gene Total UMI') 
        plt.ylabel('LOY Gene Total UMI') 
        plt.title(f'{celltype} Log Transformed LOY vs No LOY Gene Expression in Male Cells Benchmark 2000')
        plt.savefig(f'{celltype} Log LOY vs No LOY Gene Expression in Male Cells Benchmark 2000.png')
        plt.show()

        fig, ax = plt.subplots(figsize=(30,30))
        model = sns.regplot(x=lognotLOY, y=logLOY, scatter=True, fit_reg=True)
        x_regression = model.get_lines()[0].get_xdata()
        y_regression = model.get_lines()[0].get_ydata()
        plt.clf()
        # Calculate residuals
        residuals = np.array(logLOY) - np.interp(lognotLOY, x_regression, y_regression)

        sns.residplot(x=np.array(lognotLOY), y=residuals, lowess=True, line_kws={'color': 'red'})
        plt.xlabel("No LOY Gene Total UMI")
        plt.ylabel("LOY Gene Total UMI Residuals")
        label=list(all_cells_genes[notLOYdfindexlist])
        for i, residual in enumerate(residuals):
            if abs(residual) > 0.5:
                plt.annotate(label[i], (np.array(lognotLOY)[i], residual))

        plt.title(f'{celltype} Large Residual Plot Log Transformed LOY vs No LOY Gene Expression in Male Cells Benchmark 2000')
        plt.savefig(f'{celltype} Large Residual Plot Log LOY vs No LOY Gene Expression in Male Cells Benchmark 2000.png')
        plt.show()

        fig, ax = plt.subplots(figsize=(30,30))
        model = sns.regplot(x=lognotLOY, y=logLOY, scatter=True, fit_reg=True)
        x_regression = model.get_lines()[0].get_xdata()
        y_regression = model.get_lines()[0].get_ydata()
        plt.clf()
        residuals = np.array(logLOY) - np.interp(lognotLOY, x_regression, y_regression)

        sns.residplot(x=np.array(lognotLOY), y=residuals, lowess=True, line_kws={'color': 'red'})
        plt.xlabel("No LOY Gene Total UMI")
        plt.ylabel("LOY Gene Total UMI Residuals")
        label=list(all_cells_genes[notLOYdfindexlist])
        for i, residual in enumerate(residuals):
            if abs(residual) > 0.5:
                plt.annotate(label[i], (np.array(lognotLOY)[i], residual))

        plt.title(f'{celltype} Large Residual Plot Log Transformed LOY vs No LOY Gene Expression in Male Cells Benchmark 2000')
        plt.savefig(f'{celltype} Large Residual Plot Log LOY vs No LOY Gene Expression in Male Cells Benchmark 2000.png')
        plt.show()